#### import required modules

In [2]:
#scraping
from urllib.request import urlopen as ureq
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from dateutil import parser
import datetime
import time

# timeout issues
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.page_load_strategy = 'normal'
chrome_options.add_argument('--enable-automation')
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-insecure-localhost')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-browser-side-navigation')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(ChromeDriverManager().install())

#dataframes
import pandas as pd
import numpy as np
import re

#anonymize usernames
import csv
from faker import Factory
from collections import defaultdict

[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:35<00:00, 201kB/s]   
C:\Users\Owner\AppData\Local\Temp\ipykernel_17196\294728646.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
pd.set_option('display.max_columns', None)

In [2]:
#dataframes
import pandas as pd
import numpy as np
import re

#anonymize usernames
import csv
from faker import Factory
from collections import defaultdict

<a id='extract_data'></a>
## Extract Data

<a id='scraping_brand'></a>
### Scraping the local brands

In [3]:
PATH = r"D:\Downloads\Data Analitik Scrapping\chromedriver.exe"
driver = webdriver.Chrome(PATH)

brand_url ='https://reviews.femaledaily.com/brands'
#opening up connection, grabbing the page with selenium webdriver
driver.get(brand_url)

C:\Users\Owner\AppData\Local\Temp\ipykernel_13872\1866561093.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [4]:
#Indonesian brands
brand_idn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='id_tab_Indonesia']")))
driver.execute_script("arguments[0].click();", brand_idn)

#brands alphabet
brand_list = brand_idn.find_element(By.XPATH, "//div[@class='jsx-2959408550 brand-alphabet-list']")
brand_char = brand_list.find_elements(By.TAG_NAME, "a")

In [5]:
list_brand_lokal = []
sum_prod_lokal =[]
list_link = []

# list of local beauty brands
for char in brand_char:
	driver.execute_script("arguments[0].click();", char)
	brand_list_prod = char.find_element(By.XPATH, "//div[@class='jsx-2959408550 brand-list-prod']")
	brand_prod = brand_list_prod.find_elements(By.TAG_NAME, 'a')
	for name in brand_prod:
		brand_name = name.find_elements(By.TAG_NAME, 'h2')
		sum_prod = name.find_elements(By.TAG_NAME, 'p')
		list_link.append(name.get_attribute('href'))
		for only_name in brand_name:
			list_brand_lokal.append(only_name.text)
		for only_sum in sum_prod:
			sum_prod_lokal.append(only_sum.text)

In [6]:
len(list_brand_lokal)

1268

In [7]:
zip_iterator = zip(list_brand_lokal, list_link)
brand_dict = dict(zip_iterator)
dict_skincare_lokal = {}
link_product = []

In [8]:
#get list of local skincare brand & products href
for key, value in brand_dict.items():
	driver.get(value)
	try:
		skincare_filter = driver.find_element(By.XPATH, "//*[contains(text(), 'SkinCare')]")
		driver.execute_script("arguments[0].click();", skincare_filter)
		element_exist = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='jsx-4252524774 brand-product']")))
		if element_exist:
			dict_skincare_lokal[key] = value
			brand_product = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='jsx-2059197805']")))
			for product in brand_product:
				link_product.append(product.get_attribute('href'))
	except:
		print("merk " + key + " tidak punya produk skincare")

merk AIMR tidak punya produk skincare
merk ARRA Beauty tidak punya produk skincare
merk AZZURA tidak punya produk skincare
merk Absolute tidak punya produk skincare
merk Aelin Skincare tidak punya produk skincare
merk Aeluna tidak punya produk skincare
merk Aeris Beaute tidak punya produk skincare
merk Aeuko tidak punya produk skincare
merk Akiko Beauty tidak punya produk skincare
merk Alatté tidak punya produk skincare
merk Alchemist Fragrance tidak punya produk skincare
merk Aldo Akira tidak punya produk skincare
merk Alien Objects tidak punya produk skincare
merk All Is Well tidak punya produk skincare
merk Allglows tidak punya produk skincare
merk Alraiya tidak punya produk skincare
merk Alurin Beauty tidak punya produk skincare
merk Amara tidak punya produk skincare
merk Amara Kids Cosmetics tidak punya produk skincare
merk Amora tidak punya produk skincare
merk Anava'S Secret tidak punya produk skincare
merk Andalan Feminine Care tidak punya produk skincare
merk Anggie Rassly Bea

In [9]:
print(len(dict_skincare_lokal))
print(len(link_product))

781
5317


<a id='scraping_product'></a>
### Scraping the local skincare products

In [ ]:
subcategory = []
product_brand = []
product_name = []
product_variant = []
product_price =[]
product_desc = []
product_link = []

#get information for each product
for link in link_product:
	driver.get(link)
	driver.implicitly_wait(3) #wait 3 seconds

	try:
		breadcrumb_sub = driver.find_elements(By.XPATH, "//a[@class='jsx-1858436673 breadcrumb-section-text']")
		get_product_brand = driver.find_element(By.XPATH, "//h2[@class='jsx-2016320139 jsx-2462230538 product-brand']")
		get_product_name = driver.find_element(By.XPATH, "//h1[@class='jsx-2016320139 jsx-2462230538 product-name']")
        # jsx-905139883 jsx-1996910461 product-name
		get_desc_toggle = driver.find_element(By.XPATH, '//span[@class="jsx-2016320139 jsx-2462230538 toggle-desc expanded"]')
		driver.execute_script("arguments[0].click();", get_desc_toggle)
		get_product_desc = driver.find_element(By.XPATH, "//p[@class='jsx-2016320139 jsx-2462230538 product-desc']")
   
		#get product price
		try:
			get_product_price = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, "//p[@class='jsx-2016320139 jsx-2462230538 product-price']")))
			if get_product_price:
				product_price.append(get_product_price[0].text)
			else:
				product_price.append('')
		except:
			product_price.append('beauty studio')
                
		#get product variant
		get_product_variant = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, '//h3[@class="jsx-2016320139 jsx-2462230538 product-shade"]')))
		if get_product_variant:
			product_variant.append(get_product_variant[0].text)
		else:
			product_variant.append('')
	        
		#append the information to lists
		subcategory.append(breadcrumb_sub[1].text)
		product_brand.append(get_product_brand.text)
		product_name.append(get_product_name.text)
		product_desc.append(get_product_desc.text)
		product_link.append(link)
	except:
		print('something not found')

In [13]:
print(len(subcategory))
print(len(product_brand))
print(len(product_name))
print(len(product_price))
print(len(product_desc))
print(len(product_variant))

5044
5044
5044
5044
5044
5044


In [ ]:
product_df = pd.DataFrame(list(zip(subcategory, product_brand, product_name, product_variant, product_price, product_desc, product_link)), 
                  columns= ['subcategory', 'brand', 'product_name', 'variant', 'price', 'description', 'link'])

In [ ]:
product_df.to_csv("product_female_daily2.csv", index=False, header=True)

<a id='scraping_review'></a>
### Scraping the product reviews

In [4]:
product_df = pd.read_csv('product_female_daily2.csv')
product_df

,category,subcategory,brand,product_name,variant,price,description,link
0,Mask,Serum & Essence,AAGS Skincare,All in One Miracle Serum,NaN,Rp. 89.900,All in one mircale serum ini mengandung niacin...,https://reviews.femaledaily.com/products/treat...
1,Mask,Cream,AAGS Skincare,Glowing Night Cream,NaN,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",https://reviews.femaledaily.com/products/moist...
2,Mask,Cream,AAGS Skincare,Glowing Day Cream,NaN,Rp. 79.900,Glowing Day Cream ini merupakan pelembap yang ...,https://reviews.femaledaily.com/products/moist...
3,Mask,Lotion & Emulsion,AFK,Gold Gel Creme,NaN,Rp. 165.000,AFK Gold Gel Crème merupakan gold gel yang mem...,https://reviews.femaledaily.com/products/moist...
4,Mask,Acne Treatment,AFK,Acne Rejuvenation Creme,NaN,Rp. 135.000,AFK Acne Rejuvenation Crème merupakan cream ya...,https://reviews.femaledaily.com/products/treat...
...,...,...,...,...,...,...,...,...
5311,Mask,Serum & Essence,1N Perfection Skincare,Brightening B Serum,NaN,Rp. 250.000,1N PERFECTION Brightening B Serum adalah serum...,https://reviews.femaledaily.com/products/treat...
5312,Mask,Toner,1N Perfection Skincare,Clarifying Skin Tonic,NaN,Rp. 120.000,1N PERFECTION Clarifying Skin Tonic adalah ton...,https://reviews.femaledaily.com/products/clean...
5313,Mask,Sun Protection,1N Perfection Skincare,Perfect Sun Screen,NaN,Rp. 120.000,1N PERFECTION Daily Shield UV Cream Gel adalah...,https://reviews.femaledaily.com/products/moist...
5314,Mask,Facial Wash,27Degrees.id,Centella Glow Gentle Facial Wash,NaN,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,https://reviews.femaledaily.com/products/clean...


In [6]:
user_name = []
user_link = []
star_rating = []
is_recommend = []
review_date = []
user_review = []
review_link = []
product_brand = []
product_name = []
date_now = []

for link, brand, product in zip(product_df.link, product_df.brand, product_df.product_name):    
    for page in range(0,1):
        try:
            link_per_page = link[:-1] + str(page)
            driver.get(link_per_page)
            driver.implicitly_wait(3) 
            is_review_container = driver.find_element(By.XPATH, '//div[@class="jsx-2016320139 jsx-2462230538 list-reviews"]')
            review_list = driver.find_elements(By.XPATH, '//div[@class="jsx-2016320139 jsx-2462230538 item"]')
            for item in review_list:
                get_username = item.find_element(By.XPATH, './/p[@class="profile-username"]')
                get_user_link = item.find_element(By.XPATH, ".//a[contains(@href,'//account.femaledaily.com/user/fd/')]")

                star_list = item.find_element(By.XPATH, './/span[@class="cardrv-starlist"]')
                get_star_full = star_list.find_elements(By.XPATH, './/i[contains(@class,"icon-ic_big_star_full")]')
                get_star_half = star_list.find_elements(By.XPATH, './/i[contains(@class,"icon-ic_big_star_half")]')
                count_stars = len(get_star_full) + 0.5*(len(get_star_half))
                    
                get_recommendation_up = item.find_elements(By.XPATH, './/i[@class="icon-ic_thumbs_up"]')
                if get_recommendation_up:
                    recommend = 'Yes'
                else:
                    get_recommendation_down = item.find_elements(By.XPATH, './/i[@class="icon-ic_thumbs_down"]')
                    if get_recommendation_down:
                        recommend = 'No'
                    else:
                        recommend = ''
                
                get_review_text = item.find_element(By.XPATH, './/p[@class="text-content"]')
                
                get_review_date = item.find_element(By.XPATH, './/p[@class="review-date"]')

                #append the information to lists
                user_name.append(get_username.text)
                user_link.append(get_user_link.get_attribute('href'))
                star_rating.append(count_stars)
                is_recommend.append(recommend)
                user_review.append(get_review_text.text)
                review_date.append(get_review_date.get_attribute("innerHTML"))
                date_now.append(datetime.datetime.now())
                review_link.append(link_per_page)
                product_brand.append(brand)
                product_name.append(product)

        except:
            print('something not found')

something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found
something not found


In [7]:
print(len(user_name))
print(len(user_link))
print(len(star_rating))
print(len(is_recommend))
print(len(review_date))
print(len(user_review))
print(len(review_link))
print(len(product_brand))
print(len(product_name))
print(len(date_now))

32404
32404
32404
32404
32404
32404
32404
32404
32404
32404


In [8]:
review_df = pd.DataFrame(list(zip(user_name, user_link, star_rating, is_recommend, review_date, user_review,
                                  review_link, product_brand, product_name, date_now)), 
                                  columns= ['user_name', 'user_link', 'star_rating', 'is_recommend', 'review_date', 'user_review', 'review_link', 'product_brand', 'product_name', 'date_now'])

In [9]:
review_df.to_csv("detail_produk_female_daily.csv", index=False, header=True)

## Cleaning Data Scraping

In [3]:
product_df = pd.read_csv('product_without_description.csv')
product_df

,category,subcategory,brand,product_name,variant,price,link
0,Mask,Serum & Essence,AAGS Skincare,All in One Miracle Serum,NaN,Rp. 89.900,https://reviews.femaledaily.com/products/treat...
1,Mask,Cream,AAGS Skincare,Glowing Night Cream,NaN,Rp. 89.900,https://reviews.femaledaily.com/products/moist...
2,Mask,Cream,AAGS Skincare,Glowing Day Cream,NaN,Rp. 79.900,https://reviews.femaledaily.com/products/moist...
3,Mask,Lotion & Emulsion,AFK,Gold Gel Creme,NaN,Rp. 165.000,https://reviews.femaledaily.com/products/moist...
4,Mask,Acne Treatment,AFK,Acne Rejuvenation Creme,NaN,Rp. 135.000,https://reviews.femaledaily.com/products/treat...
...,...,...,...,...,...,...,...
5311,Mask,Serum & Essence,1N Perfection Skincare,Brightening B Serum,NaN,Rp. 250.000,https://reviews.femaledaily.com/products/treat...
5312,Mask,Toner,1N Perfection Skincare,Clarifying Skin Tonic,NaN,Rp. 120.000,https://reviews.femaledaily.com/products/clean...
5313,Mask,Sun Protection,1N Perfection Skincare,Perfect Sun Screen,NaN,Rp. 120.000,https://reviews.femaledaily.com/products/moist...
5314,Mask,Facial Wash,27Degrees.id,Centella Glow Gentle Facial Wash,NaN,Rp. 85.000,https://reviews.femaledaily.com/products/clean...


In [5]:
review_df = pd.read_csv('detail_produk_female_daily.csv')
review_df

,user_name,user_link,star_rating,is_recommend,review_date,user_review,review_link,product_brand,product_name,date_now
0,wulanraahma,https://account.femaledaily.com/user/fd/wulanr...,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,https://reviews.femaledaily.com/products/treat...,AAGS Skincare,All in One Miracle Serum,2023-04-29 16:16:17.793512
1,sofiahdynt,https://account.femaledaily.com/user/fd/sofiah...,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",https://reviews.femaledaily.com/products/treat...,AAGS Skincare,All in One Miracle Serum,2023-04-29 16:16:21.095234
2,wulanraahma,https://account.femaledaily.com/user/fd/wulanr...,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,https://reviews.femaledaily.com/products/moist...,AAGS Skincare,Glowing Night Cream,2023-04-29 16:16:28.235663
3,sofiahdynt,https://account.femaledaily.com/user/fd/sofiah...,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,https://reviews.femaledaily.com/products/moist...,AAGS Skincare,Glowing Night Cream,2023-04-29 16:16:31.534267
4,popy_mercylin,https://account.femaledaily.com/user/fd/popy_m...,5.0,Yes,11 Feb 2023,"Aku suka banget sama wangi nya… aromanya enak,...",https://reviews.femaledaily.com/products/moist...,AAGS Skincare,Glowing Day Cream,2023-04-29 16:16:38.103850
...,...,...,...,...,...,...,...,...,...,...
32399,farahn27,https://account.femaledaily.com/user/fd/farahn27,5.0,Yes,06 Oct 2021,Menurutku ini cleanser paling nyaman krn ga ad...,https://reviews.femaledaily.com/products/clean...,27Degrees.id,Centella Glow Gentle Facial Wash,2023-05-01 10:43:28.038655
32400,kitchendw,https://account.femaledaily.com/user/fd/kitchendw,3.0,Yes,17 Jun 2021,"ini kurang suka sih, karna dia gada busa sama ...",https://reviews.femaledaily.com/products/clean...,27Degrees.id,Centella Glow Gentle Facial Wash,2023-05-01 10:43:31.250034
32401,RosmaCindy,https://account.femaledaily.com/user/fd/RosmaC...,5.0,Yes,29 May 2021,ini cocok banget buat kulit sensitif kaya kuli...,https://reviews.femaledaily.com/products/clean...,27Degrees.id,Centella Glow Gentle Facial Wash,2023-05-01 10:43:34.408773
32402,Indahpspta_,https://account.femaledaily.com/user/fd/Indahp...,5.0,Yes,27 May 2021,27degrees facial wash inii bagus bangetttttt. ...,https://reviews.femaledaily.com/products/clean...,27Degrees.id,Centella Glow Gentle Facial Wash,2023-05-01 10:43:37.625002


In [6]:
df_gabungan = pd.merge(product_df, review_df, on='product_name')
# df_gabungan.to_csv('data_scraping_female_daily.csv', index=False)

In [7]:
df = df_gabungan.drop(columns=['variant', 'link', 'user_link', 'review_link', 'date_now', 'brand'])
df 

,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera
...,...,...,...,...,...,...,...,...,...,...,...
82382,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,nfanlyy,5.0,Yes,28 Jan 2022,sabun muka ini tuh nyaman bgt di aplikasiin ke...,27Degrees.id
82383,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,farahn27,5.0,Yes,06 Oct 2021,Menurutku ini cleanser paling nyaman krn ga ad...,27Degrees.id
82384,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,kitchendw,3.0,Yes,17 Jun 2021,"ini kurang suka sih, karna dia gada busa sama ...",27Degrees.id
82385,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,RosmaCindy,5.0,Yes,29 May 2021,ini cocok banget buat kulit sensitif kaya kuli...,27Degrees.id


In [8]:
df.duplicated().sum()

5032

In [9]:
df = df.drop_duplicates().reset_index(drop=True)
df

,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera
...,...,...,...,...,...,...,...,...,...,...,...
77350,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,nfanlyy,5.0,Yes,28 Jan 2022,sabun muka ini tuh nyaman bgt di aplikasiin ke...,27Degrees.id
77351,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,farahn27,5.0,Yes,06 Oct 2021,Menurutku ini cleanser paling nyaman krn ga ad...,27Degrees.id
77352,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,kitchendw,3.0,Yes,17 Jun 2021,"ini kurang suka sih, karna dia gada busa sama ...",27Degrees.id
77353,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,RosmaCindy,5.0,Yes,29 May 2021,ini cocok banget buat kulit sensitif kaya kuli...,27Degrees.id


In [10]:
df.to_csv('data_scraping_female_daily_with_desc.csv', index=False, header=True)

<a id='cleaning_product'></a>
### Cleaning Product Dataset

In [11]:
product_df = pd.read_csv('data_scraping_female_daily_with_desc.csv')
product_df.head(5)

,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera


In [12]:
product_df.shape

(77355, 11)

In [13]:
product_df.dtypes

category          object
subcategory       object
product_name      object
price             object
description       object
user_name         object
star_rating      float64
is_recommend      object
review_date       object
user_review       object
product_brand     object
dtype: object

In [14]:
product_df.duplicated().sum()

0

In [15]:
product_df.isna().sum()

category             0
subcategory          0
product_name         0
price                0
description          0
user_name            0
star_rating          0
is_recommend     11022
review_date          0
user_review          0
product_brand        0
dtype: int64

In [16]:
product_df = product_df.dropna().reset_index(drop=True)
product_df

,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera
...,...,...,...,...,...,...,...,...,...,...,...
66328,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,nfanlyy,5.0,Yes,28 Jan 2022,sabun muka ini tuh nyaman bgt di aplikasiin ke...,27Degrees.id
66329,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,farahn27,5.0,Yes,06 Oct 2021,Menurutku ini cleanser paling nyaman krn ga ad...,27Degrees.id
66330,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,kitchendw,3.0,Yes,17 Jun 2021,"ini kurang suka sih, karna dia gada busa sama ...",27Degrees.id
66331,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,RosmaCindy,5.0,Yes,29 May 2021,ini cocok banget buat kulit sensitif kaya kuli...,27Degrees.id


<a id='cleaning_review'></a>
### Cleaning Review Dataset

In [17]:
product_df = product_df.replace('', np.nan)

In [18]:
product_df.isnull().sum()

category         0
subcategory      0
product_name     0
price            0
description      0
user_name        0
star_rating      0
is_recommend     0
review_date      0
user_review      0
product_brand    0
dtype: int64

In [19]:
product_df.duplicated().sum()

0

In [20]:
product_df.to_csv("data_scraping_clean.csv", index=False, header=True)
product_df

,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Mask,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Mask,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera
...,...,...,...,...,...,...,...,...,...,...,...
66328,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,nfanlyy,5.0,Yes,28 Jan 2022,sabun muka ini tuh nyaman bgt di aplikasiin ke...,27Degrees.id
66329,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,farahn27,5.0,Yes,06 Oct 2021,Menurutku ini cleanser paling nyaman krn ga ad...,27Degrees.id
66330,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,kitchendw,3.0,Yes,17 Jun 2021,"ini kurang suka sih, karna dia gada busa sama ...",27Degrees.id
66331,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,RosmaCindy,5.0,Yes,29 May 2021,ini cocok banget buat kulit sensitif kaya kuli...,27Degrees.id


In [3]:
# Membaca file CSV
df = pd.read_csv('data_scraping_clean.csv')

# Mengganti nilai "face" menjadi "makeup remover" dalam kolom subcategory
df.loc[df['subcategory'] == 'Face', 'subcategory'] = 'Makeup Remover'

# Menyimpan hasil perubahan ke file CSV baru
df.to_csv('data_scraping_clean.csv', index=False)

In [10]:
# Menghitung jumlah field duplikat pada variabel "review"
jumlah_duplikat = df.duplicated(subset=['user_review']).sum()

# Menampilkan jumlah field duplikat pada variabel "review"
print("Jumlah field duplikat pada variabel review: ", jumlah_duplikat)

Jumlah field duplikat pada variabel review:  37330


In [11]:
df = df.drop_duplicates(subset=['user_review'])
df

,category,subcategory,product_name,price,description,user_name,star_rating,is_recommend,review_date,user_review,product_brand
0,Treatment,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,wulanraahma,5.0,Yes,07 Oct 2021,Teksturnya runny gel dan ada edible gold nya. ...,AAGS Skincare
1,Treatment,Serum & Essence,All in One Miracle Serum,Rp. 89.900,All in one mircale serum ini mengandung niacin...,sofiahdynt,5.0,Yes,03 Sep 2021,"Teksturnya gel like, agak slimey, dan mudah di...",AAGS Skincare
2,Moisturizer,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",wulanraahma,5.0,Yes,07 Oct 2021,Untuk hasilnya yang aku rasain itu ngelembapin...,AAGS Skincare
3,Moisturizer,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",sofiahdynt,4.0,Yes,03 Sep 2021,Teksturnya gel cream berwarna kuning muda deng...,AAGS Skincare
4,Moisturizer,Cream,Glowing Night Cream,Rp. 89.900,"Night cream ini mengandung cucumber extract, n...",popy_mercylin,5.0,Yes,19 Feb 2023,night cream ini salah satu cream yang jadi fav...,Adera
...,...,...,...,...,...,...,...,...,...,...,...
66328,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,nfanlyy,5.0,Yes,28 Jan 2022,sabun muka ini tuh nyaman bgt di aplikasiin ke...,27Degrees.id
66329,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,farahn27,5.0,Yes,06 Oct 2021,Menurutku ini cleanser paling nyaman krn ga ad...,27Degrees.id
66330,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,kitchendw,3.0,Yes,17 Jun 2021,"ini kurang suka sih, karna dia gada busa sama ...",27Degrees.id
66331,Mask,Facial Wash,Centella Glow Gentle Facial Wash,Rp. 85.000,27Degrees Centella Glow Gentle Facial Wash mer...,RosmaCindy,5.0,Yes,29 May 2021,ini cocok banget buat kulit sensitif kaya kuli...,27Degrees.id


In [ ]:
df.to_csv('data_clean_analytics_femaledaily.csv', index=False)

#### impute is_recomend missing value

In [4]:
df.is_recommend.value_counts()

Yes    55674
No     10659
Name: is_recommend, dtype: int64

In [5]:
df.is_recommend.isnull().sum()

0

In [54]:
product_df.groupby(['star_rating', 'is_recommend'], dropna=False, as_index=False).size()

,star_rating,is_recommend,size
0,1.0,No,1345
1,1.0,Yes,47
2,2.0,No,2407
3,2.0,Yes,235
4,3.0,No,3235
5,3.0,Yes,3219
6,4.0,No,532
7,4.0,Yes,13988
8,5.0,No,29
9,5.0,Yes,28341
